In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
banknote_data = pd.read_csv('../data/data_banknote_authentication.csv')
banknote_data.head()

,Variance,Skewness,curtosis,entropy,target
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [18]:
X = banknote_data.drop(['target'], axis=1)
y = banknote_data[['target']]

In [19]:
banknote_data['target'].value_counts()

0    762
1    610
Name: target, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [21]:
print(X_train.shape)
print(X_test.shape)

(960, 4)
(412, 4)


In [23]:
X_train_T = X_train.copy()
X_train_T['target'] = y_train
X_train_T = X_train_T[X_train_T['target']==0]
X_train_T = X_train_T.drop('target', axis=1)
X_train_T = np.asarray(X_train_T).astype(np.float32)

X_test_T = X_test.copy()
X_test_T['target'] = y_test
X_test_T = X_test_T[X_test_T['target']==0]
X_test_T = X_test_T.drop('target', axis=1)
X_test_T = np.asarray(X_test_T).astype(np.float32)

In [27]:

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, LSTM, Dropout, Dense, Flatten, Embedding, Conv1D, Input
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, classification_report, auc, roc_curve, accuracy_score

epochs = 400
batch_size = 8
input_dim = X_train_T.shape[1]
encoding_dim = 2
def autoencoder_model():
    input_layer = Input(shape=(input_dim,))
    encoder_layer = Dense(encoding_dim, activation='relu')(input_layer)
    decoder_layer = Dense(input_dim, activation='linear')(encoder_layer)
    autoencoder = Model(inputs=input_layer, outputs=decoder_layer)

    encoder = Model(inputs=input_layer, outputs=encoder_layer)
    return autoencoder

model = autoencoder_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 12        
Total params: 22
Trainable params: 22
Non-trainable params: 0
_________________________________________________________________


In [31]:
history = model.fit(X_train_T, X_train_T, batch_size=batch_size, epochs=epochs,
                    shuffle=True,
                    validation_data=(X_test_T, X_test_T),
                    verbose=1)


Epoch 1/400
67/67 [==============================] - 1s 3ms/step - loss: 33.5848 - acc: 0.1299 - val_loss: 31.6514 - val_acc: 0.1853
Epoch 2/400
67/67 [==============================] - 0s 1ms/step - loss: 26.7835 - acc: 0.3222 - val_loss: 25.4814 - val_acc: 0.4397
Epoch 3/400
67/67 [==============================] - 0s 1ms/step - loss: 22.7555 - acc: 0.5609 - val_loss: 20.9774 - val_acc: 0.5948
Epoch 4/400
67/67 [==============================] - 0s 1ms/step - loss: 17.9521 - acc: 0.6021 - val_loss: 17.4271 - val_acc: 0.6422
Epoch 5/400
67/67 [==============================] - 0s 1ms/step - loss: 15.6886 - acc: 0.6720 - val_loss: 14.5946 - val_acc: 0.6681
Epoch 6/400
67/67 [==============================] - 0s 1ms/step - loss: 12.4390 - acc: 0.6923 - val_loss: 12.4331 - val_acc: 0.6853
Epoch 7/400
67/67 [==============================] - 0s 1ms/step - loss: 10.8706 - acc: 0.6956 - val_loss: 10.8249 - val_acc: 0.7112
Epoch 8/400
67/67 [==============================] - 0s 1ms/step - lo

67/67 [==============================] - 0s 1ms/step - loss: 1.4226 - acc: 0.8715 - val_loss: 1.5439 - val_acc: 0.9267
Epoch 64/400
67/67 [==============================] - 0s 1ms/step - loss: 1.3377 - acc: 0.9144 - val_loss: 1.5095 - val_acc: 0.9267
Epoch 65/400
67/67 [==============================] - 0s 1ms/step - loss: 1.4038 - acc: 0.9287 - val_loss: 1.4764 - val_acc: 0.9267
Epoch 66/400
67/67 [==============================] - 0s 1ms/step - loss: 1.3707 - acc: 0.9224 - val_loss: 1.4452 - val_acc: 0.9267
Epoch 67/400
67/67 [==============================] - 0s 1ms/step - loss: 1.1977 - acc: 0.9338 - val_loss: 1.4162 - val_acc: 0.9310
Epoch 68/400
67/67 [==============================] - 0s 1ms/step - loss: 1.3441 - acc: 0.9330 - val_loss: 1.3884 - val_acc: 0.9310
Epoch 69/400
67/67 [==============================] - 0s 1ms/step - loss: 1.1718 - acc: 0.9404 - val_loss: 1.3617 - val_acc: 0.9310
Epoch 70/400
67/67 [==============================] - 0s 1ms/step - loss: 1.2257 - acc: 0

Epoch 125/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7609 - acc: 0.9520 - val_loss: 0.8723 - val_acc: 0.9612
Epoch 126/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7632 - acc: 0.9222 - val_loss: 0.8646 - val_acc: 0.9612
Epoch 127/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7822 - acc: 0.9356 - val_loss: 0.8624 - val_acc: 0.9612
Epoch 128/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7788 - acc: 0.9454 - val_loss: 0.8627 - val_acc: 0.9612
Epoch 129/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7264 - acc: 0.9261 - val_loss: 0.8591 - val_acc: 0.9612
Epoch 130/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7487 - acc: 0.9377 - val_loss: 0.8573 - val_acc: 0.9655
Epoch 131/400
67/67 [==============================] - 0s 1ms/step - loss: 0.8404 - acc: 0.9340 - val_loss: 0.8553 - val_acc: 0.9612
Epoch 132/400
67/67 [==============================] - 0s 1ms/step - 

67/67 [==============================] - 0s 1ms/step - loss: 0.6672 - acc: 0.9447 - val_loss: 0.7896 - val_acc: 0.9655
Epoch 187/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7190 - acc: 0.9271 - val_loss: 0.7890 - val_acc: 0.9655
Epoch 188/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7589 - acc: 0.9424 - val_loss: 0.7923 - val_acc: 0.9655
Epoch 189/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6594 - acc: 0.9561 - val_loss: 0.7863 - val_acc: 0.9655
Epoch 190/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7664 - acc: 0.9534 - val_loss: 0.7887 - val_acc: 0.9655
Epoch 191/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6886 - acc: 0.9518 - val_loss: 0.7860 - val_acc: 0.9698
Epoch 192/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6902 - acc: 0.9257 - val_loss: 0.7846 - val_acc: 0.9655
Epoch 193/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6981 -

Epoch 248/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7326 - acc: 0.9666 - val_loss: 0.7523 - val_acc: 0.9698
Epoch 249/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6166 - acc: 0.9590 - val_loss: 0.7491 - val_acc: 0.9741
Epoch 250/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6509 - acc: 0.9655 - val_loss: 0.7487 - val_acc: 0.9698
Epoch 251/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7313 - acc: 0.9450 - val_loss: 0.7492 - val_acc: 0.9741
Epoch 252/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6737 - acc: 0.9623 - val_loss: 0.7496 - val_acc: 0.9698
Epoch 253/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6356 - acc: 0.9678 - val_loss: 0.7477 - val_acc: 0.9741
Epoch 254/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6733 - acc: 0.9535 - val_loss: 0.7484 - val_acc: 0.9698
Epoch 255/400
67/67 [==============================] - 0s 1ms/step - 

67/67 [==============================] - 0s 1ms/step - loss: 0.6721 - acc: 0.9578 - val_loss: 0.7400 - val_acc: 0.9784
Epoch 310/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6110 - acc: 0.9624 - val_loss: 0.7395 - val_acc: 0.9741
Epoch 311/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6365 - acc: 0.9726 - val_loss: 0.7389 - val_acc: 0.9784
Epoch 312/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6591 - acc: 0.9616 - val_loss: 0.7401 - val_acc: 0.9784
Epoch 313/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6109 - acc: 0.9648 - val_loss: 0.7383 - val_acc: 0.9784
Epoch 314/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6144 - acc: 0.9610 - val_loss: 0.7397 - val_acc: 0.9828
Epoch 315/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6828 - acc: 0.9776 - val_loss: 0.7383 - val_acc: 0.9828
Epoch 316/400
67/67 [==============================] - 0s 1ms/step - loss: 0.7087 -

Epoch 371/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6626 - acc: 0.9666 - val_loss: 0.7357 - val_acc: 0.9828
Epoch 372/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6133 - acc: 0.9817 - val_loss: 0.7370 - val_acc: 0.9871
Epoch 373/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6507 - acc: 0.9612 - val_loss: 0.7358 - val_acc: 0.9828
Epoch 374/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6308 - acc: 0.9723 - val_loss: 0.7366 - val_acc: 0.9828
Epoch 375/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6504 - acc: 0.9619 - val_loss: 0.7371 - val_acc: 0.9828
Epoch 376/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6705 - acc: 0.9685 - val_loss: 0.7366 - val_acc: 0.9828
Epoch 377/400
67/67 [==============================] - 0s 1ms/step - loss: 0.6465 - acc: 0.9508 - val_loss: 0.7367 - val_acc: 0.9871
Epoch 378/400
67/67 [==============================] - 0s 1ms/step - 

In [32]:
X_test_prod = model.predict(X_test)

In [34]:
mean_seq_error = np.mean(np.power(X_test-X_test_prod, 2), axis=1)